<img src="https://app.powerbi.com/13.0.10537.111/images/PowerBI96x96.png?raw=true" width="10%"> ➕
<img src="https://github.com/Microsoft/azuredatastudio/blob/master/samples/notebookSamples/Graphics/AzureDataStudioLogo.png?raw=true" width="10%"> = ❤

# Collect Power Platform Data Gateway information

This Notebook will walk you through how to collect:
+ All of the Data Gateway Clusters within your tenant
+ All of the Member servers of thoseData Gateway Clusters
+ All of the Data Sources on each of those Clusters
+ All Users of those Data Sources

NOTE: This Notebook exports all the this information to different Worksheets within the same Excel file.  In order to do this, we use the community-driven `ImportExcel` module from the PowerShell Gallery.  Also of note, the name of the Excel file we are exporting to is named `c:\temp\PPGateways.xlsx` but you may want to change that.  Alternatively, you could modify the code below to export the results to individual .CSV files.


Install the `DataGateway` module from the PowerShell Gallery:

In [ ]:
Install-Module DataGateway

Before you can use any of the cmdlets in the `DataGateway` module, you must first login to your tenant using the `Login-DataGateway` command.

In [1]:
Login-DataGateway

Collect all Data Gateway Clusters

In [2]:
Get-DataGatewayCluster -Scope Organization |
Export-Excel -Path c:\temp\PPGateways.xlsx -WorksheetName DataGatewayClusters

Collect info about each Member of your Data Gateway Clusters

In [ ]:
Get-DataGatewayCluster -Scope Organization | SELECT -ExpandProperty MemberGateways |
Export-Excel -Path c:\temp\PPGateways.xlsx -WorksheetName ClusterMembers

Collect the info about each Data Source within each Data Gateway Cluster

In [ ]:
Get-DataGatewayCluster | Get-DataGatewayClusterDatasource |
Export-Excel -Path c:\temp\PPGateways.xlsx -WorksheetName ClusterDatasource

Collect all of the users of the above Data Sources

In [ ]:
foreach ($clusterDS in Get-DataGatewayCluster | Get-DataGatewayClusterDatasource) 
{
     $a = Get-DataGatewayClusterDatasourceUser -GatewayClusterId $clusterDS.ClusterId -GatewayClusterDatasourceId $clusterDS.Id
     $a | Add-Member -NotePropertyName ClusterId -NotePropertyValue $clusterDS.ClusterId; 
     $a | Add-Member -NotePropertyName DatasourceId -NotePropertyValue $clusterDS.Id; 
     $a | Export-Excel -Path c:\temp\PPGateways.xlsx -WorksheetName DatasourcePermissions -Append
}

Once your done, you can import the `PPGateways.xlsx` file into your Power BI Report.